# music 21 테스트 해보기

In [1]:
from music21 import converter, instrument, note, chord
import glob    # 원문에는 없지만 아래에서 사용하기 때문에 glob 을 import 해줘야합니다.

notes = []
for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    try:       # 학습 데이터 중 TypeError 를 일으키는 파일이 있어서 해놓은 예외처리
        parts = instrument.partitionByInstrument(midi)
    except TypeError:
        print('## 1 {} file occur error.'.format(file))
    if parts: # file has instrument parts
        print('## 2 {} file has instrument parts'.format(file))
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        print('## 3 {} file has notes in a flat structure'.format(file))
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

## 2 midi_songs/pkelite4.mid file has instrument parts
## 2 midi_songs/OTD5YA.mid file has instrument parts
## 2 midi_songs/path_of_repentance.mid file has instrument parts
## 2 midi_songs/Rachel_Piano_tempofix.mid file has instrument parts


In [2]:
glob.glob("midi_songs/*.mid")

['midi_songs/pkelite4.mid',
 'midi_songs/OTD5YA.mid',
 'midi_songs/path_of_repentance.mid',
 'midi_songs/Rachel_Piano_tempofix.mid']

In [3]:
parts = instrument.partitionByInstrument(midi)
len(parts.parts)

1

In [4]:
notes_to_parse = parts.parts[0].recurse()
notes_to_parse

<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>

In [5]:
for element in notes_to_parse:
    print(element)

<music21.stream.Part Piano>
Piano
<music21.tempo.MetronomeMark presto Quarter=184.0>
<music21.meter.TimeSignature 4/4>
<music21.meter.TimeSignature 4/4>
<music21.note.Note G>
<music21.note.Note G>
<music21.note.Rest rest>
<music21.note.Note B>
<music21.note.Rest rest>
<music21.note.Note E>
<music21.note.Rest rest>
<music21.note.Rest rest>
<music21.note.Note G>
<music21.note.Note G>
<music21.note.Rest rest>
<music21.note.Rest rest>
<music21.tempo.MetronomeMark presto Quarter=184.0>
<music21.note.Note F#>
<music21.note.Note F#>
<music21.note.Rest rest>
<music21.note.Rest rest>
<music21.note.Note E>
<music21.note.Note E>
<music21.note.Rest rest>
<music21.note.Rest rest>
<music21.note.Note D>
<music21.note.Note D>
<music21.note.Rest rest>
<music21.note.Rest rest>
<music21.note.Note F#>
<music21.note.Note F#>
<music21.note.Rest rest>
<music21.note.Rest rest>
<music21.note.Note E>
<music21.note.Note E>
<music21.note.Rest rest>
<music21.note.Rest rest>
<music21.note.Note C>
<music21.note.Note

# 케라스 LSTM 모델로 작곡하기

#### 필요한 라이브러리 import

In [6]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, instrument, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 전처리 : midi 파일포맷의 음악을 note와 코드(chord) 로 표현하기

In [7]:
import os
import pickle

def get_notes():
    """midi_songs 디렉토리에서 모든 midi 파일을 열어서 모든 노트와 코드(chord)를 가져오는 함수."""
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    if not os.path.exists('data'):
        os.makedirs('data')
        
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [8]:
notes = get_notes()

Parsing midi_songs/pkelite4.mid
Parsing midi_songs/OTD5YA.mid
Parsing midi_songs/path_of_repentance.mid
Parsing midi_songs/Rachel_Piano_tempofix.mid


### 입력 데이터 만들기

In [9]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # 모든 계이름의 이름을 pitchnames 변수에 저장.
    # set 으로 중복을 피하고, sorted 함수로 sorting 함.
    pitchnames = sorted(set(item for item in notes))

    # 각 계이름을 숫자로 바꾸는 dictionary(사전)을 만든다.
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # 입력 시퀀스를 만든다.
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # 데이터 입력 형태를 LSTM 레이어에 알맞게 변경함.
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # 입력값을 normalizing(정규화)
    network_input = network_input / float(n_vocab)
    
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

### LSTM 모델 만들기

In [10]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

### 만든 모델을 준비된 학습데이터로 학습하기

##### 모델을 학습할 시간이 충분하다면 아래 코드로 학습을 하고, 그렇지 않다면 다음 다음 셀로 학습된 모델을 사용하세요!

In [11]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=50, batch_size=64, callbacks=callbacks_list)

In [12]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [13]:
if __name__ == '__main__':
    train_network()

Parsing midi_songs/pkelite4.mid
Parsing midi_songs/OTD5YA.mid
Parsing midi_songs/path_of_repentance.mid
Parsing midi_songs/Rachel_Piano_tempofix.mid
Epoch 1/50
1162/1162 [==============================] - 72s 62ms/step - loss: 4.2713
Epoch 2/50
1162/1162 [==============================] - 63s 54ms/step - loss: 3.7613
Epoch 3/50
1162/1162 [==============================] - 67s 58ms/step - loss: 3.7413
Epoch 4/50
1162/1162 [==============================] - 67s 58ms/step - loss: 3.6979
Epoch 5/50
1162/1162 [==============================] - 66s 57ms/step - loss: 3.6853
Epoch 6/50
1162/1162 [==============================] - 64s 55ms/step - loss: 3.6800
Epoch 7/50
1162/1162 [==============================] - 66s 57ms/step - loss: 3.6831
Epoch 8/50
1162/1162 [==============================] - 67s 57ms/step - loss: 3.6699
Epoch 9/50
1162/1162 [==============================] - 66s 57ms/step - loss: 3.6679
Epoch 10/50
1162/1162 [==============================] - 67s 58ms/step - loss: 3.6606


### 음악 생성 모델 만들기

In [20]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # 각각의 뉴런(노드)의 가중치를 로드합니다.
    # 파일에 저장한 학습 결과를 가져오는 것과 같습니다!
    # model.load_weights('weights.hdf5')
    model.load_weights('weights-improvement-47-3.6276-bigger.hdf5')

    return model

### 음악 생성하기

In [21]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [22]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [23]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # 입력 시퀀스를 랜덤하게 주는 부분.
    start = numpy.random.randint(0, len(network_input)-1)

    # 숫자를 노트로 맵핑하는 사전을 생성합니다.
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # 500 개의 노트를 만들어줍니다.
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        # 입력 값에 대해 다음 노트를 예측합니다.
        prediction = model.predict(prediction_input, verbose=0)
        # 결과값은 숫자가 아닌 노트여야 하므로, 미리 만들어놓은 사전에 숫자를 넣어서 맵핑시킵니다.
        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [24]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # 모델에 의해 예측된 값을 바탕으로 노트와 코드(chord) 객체를 만듭니다.
    for pattern in prediction_output:
        # 패턴이(출력값이) 코드(chord) 일 때
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # 패턴이(출력값이) 노트일 때
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # 각 반복마 오프셋을 0.5 씩 증가시켜 줍니다.
        # 그렇지 않으면 같은 오프셋에 음이 쌓이게 됩니다.
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

In [25]:
generate()